<a href="https://colab.research.google.com/github/saisuresh1999/Assignment/blob/master/Assignment_6_Estimators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

Install and import all the necessary libraries for the assignment.

In [0]:
!pip install tensorflow==2.0.0-rc0

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns_colors = sns.color_palette('colorblind')

tf.random.set_seed(1)

### **Importing the dataset**

In [0]:
boston_dataset = load_boston()

data_X = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data_Y = pd.DataFrame(boston_dataset.target, columns=["target"])
data = pd.concat([data_X, data_Y], axis=1)

In [0]:
train, test = train_test_split(data, test_size=0.2, random_state=1)
print(len(train), "train examples")
print(len(test), "test examples")
train.head()

Converting the Pandas DataFrames into Tensorflow Datasets

In [0]:
def df_to_dataset(dataframe, shuffle_and_repeat=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle_and_repeat:
    ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.repeat()
    
  ds = ds.batch(batch_size)
  
  return ds

### Defining Feature Columns

In [0]:
# define feature_columns as a list of all the features as numeric columns using functions from tf.feature_column
feature_columns = []
for header in ['CRIM',	'ZN',	'INDUS',	'CHAS',	'NOX',	'RM',	'AGE',	'DIS',	'RAD',	'TAX',	'PTRATIO',	'B',	'LSTAT'	]:
  feature_columns.append(tf.feature_column.numeric_column(header))


### Building the model

In [0]:
config = tf.estimator.RunConfig(tf_random_seed=1)


# Build the model using tf.estimator and pass as config variable to the config parameter of the estimator
regressor = tf.estimator.BoostedTreesRegressor(
    feature_columns=feature_columns,
    n_batches_per_layer =1,
    center_bias=True,
    config=config)

Train the model by using the df_to_dataset function. Batch size should be 32. Note that while training the model, the shuffle_and_repeat parameter of the df_to_dataset function should be set to True.

In [0]:
# Train the model
regressor.train(
 input_fn=lambda: df_to_dataset(train),
    steps=50)


In [0]:
# Evaluate the model using the df_to_dataset function and the test dataframe
eval_result = regressor.evaluate(
    input_fn=lambda: df_to_dataset(test, shuffle_and_repeat=False))



In [0]:
eval_result = regressor.evaluate(
    input_fn=lambda: df_to_dataset(train, shuffle_and_repeat=False))

### **Boosted Trees Interpretation**

You will need this section only for questions 5-7.

In [0]:
pred_dicts = list(regressor.experimental_predict_with_explanations(lambda: df_to_dataset(test, False, 32)))

In [0]:
# Create DFC Pandas dataframe.
predictions = pd.Series([pred['predictions'][0] for pred in pred_dicts])
df_dfc = pd.DataFrame([pred['dfc'] for pred in pred_dicts])
df_dfc.describe().T

In [0]:
# Sum of DFCs + bias == probabality.
bias = pred_dicts[0]['bias']
dfc_pred = df_dfc.sum(axis=1) + bias
np.testing.assert_almost_equal(dfc_pred.values,
                               predictions.values)

In [0]:
# Boilerplate code for plotting :)
def _get_color(value):
    """To make positive DFCs plot green, negative DFCs plot red."""
    green, red = sns.color_palette()[2:4]
    if value >= 0: return green
    return red

def _add_feature_values(feature_values, ax):
    """Display feature's values on left of plot."""
    x_coord = ax.get_xlim()[0]
    OFFSET = 0.15
    for y_coord, (feat_name, feat_val) in enumerate(feature_values.items()):
        t = plt.text(x_coord, y_coord - OFFSET, '{}'.format(feat_val), size=12)
        t.set_bbox(dict(facecolor='white', alpha=0.5))
    from matplotlib.font_manager import FontProperties
    font = FontProperties()
    font.set_weight('bold')
    t = plt.text(x_coord, y_coord + 1 - OFFSET, 'feature\nvalue',
    fontproperties=font, size=12)

def plot_example(example):
  TOP_N = 8 # View top 8 features.
  sorted_ix = example.abs().sort_values()[-TOP_N:].index  # Sort by magnitude.
  example = example[sorted_ix]
  colors = example.map(_get_color).tolist()
  ax = example.to_frame().plot(kind='barh',
                          color=[colors],
                          legend=None,
                          alpha=0.75,
                          figsize=(10,6))
  ax.grid(False, axis='y')
  ax.set_yticklabels(ax.get_yticklabels(), size=14)

  # Add feature values.
  _add_feature_values(test.iloc[ID][sorted_ix], ax)
  return ax

In [0]:
# Plot results.
ID = 10
example = df_dfc.iloc[ID]  # Choose ith example from evaluation set.
TOP_N = 8  # View top 8 features.
sorted_ix = example.abs().sort_values()[-TOP_N:].index
ax = plot_example(example)
ax.set_title('Feature contributions for example {}\n pred: {:1.2f};'.format(ID, predictions[ID]))
ax.set_xlabel('Contribution to prediction', size=14)
plt.show()

In [0]:
# Boilerplate plotting code.
def dist_violin_plot(df_dfc, ID):
  # Initialize plot.
  fig, ax = plt.subplots(1, 1, figsize=(10, 6))

  # Create example dataframe.
  TOP_N = 8  # View top 8 features.
  example = df_dfc.iloc[ID]
  ix = example.abs().sort_values()[-TOP_N:].index
  example = example[ix]
  example_df = example.to_frame(name='dfc')

  # Add contributions of entire distribution.
  parts=ax.violinplot([df_dfc[w] for w in ix],
                 vert=False,
                 showextrema=False,
                 widths=0.7,
                 positions=np.arange(len(ix)))
  face_color = sns_colors[0]
  alpha = 0.15
  for pc in parts['bodies']:
      pc.set_facecolor(face_color)
      pc.set_alpha(alpha)

  # Add feature values.
  _add_feature_values(test.iloc[ID][sorted_ix], ax)

  # Add local contributions.
  ax.scatter(example,
              np.arange(example.shape[0]),
              color=sns.color_palette()[2],
              s=100,
              marker="s",
              label='contributions for example')

  # Legend
  # Proxy plot, to show violinplot dist on legend.
  ax.plot([0,0], [1,1], label='eval set contributions\ndistributions',
          color=face_color, alpha=alpha, linewidth=10)
  legend = ax.legend(loc='lower right', shadow=True, fontsize='x-large',
                     frameon=True)
  legend.get_frame().set_facecolor('white')

  # Format plot.
  ax.set_yticks(np.arange(example.shape[0]))
  ax.set_yticklabels(example.index)
  ax.grid(False, axis='y')
  ax.set_xlabel('Contribution to prediction', size=14)

In [0]:
dist_violin_plot(df_dfc, ID)
plt.title('Feature contributions for example {}\n pred: {:1.2f}'.format(ID, predictions[ID]))
plt.show()

### Gain-based feature importances

In [0]:
importances = regressor.experimental_feature_importances(normalize=True)
df_imp = pd.Series(importances)

# Visualize importances.
N = 8
ax = (df_imp.iloc[0:N][::-1]
    .plot(kind='barh',
          color=sns_colors[0],
          title='Gain feature importances',
          figsize=(10, 6)))
ax.grid(False, axis='y')

You can also see how DFCs vary as a feature value varies.

In [0]:
FEATURE = 'LSTAT'
feature = pd.Series(df_dfc[FEATURE].values, index=test[FEATURE].values).sort_index()
ax = sns.regplot(feature.index.values, feature.values, lowess=True)
ax.set_ylabel('contribution')
ax.set_xlabel(FEATURE)
ax.set_xlim(-35, 35)
plt.show()